# Work 1 

In [6]:
%cd /Users/trancatkhanh/Downloads/GAME

/Users/trancatkhanh/Downloads/GAME


In [3]:
from base.Splendor_v3.env import *

import numpy as np
bias1 = [np.random.rand(getActionSize()),np.zeros(getActionSize())]

print(bias1)

[array([0.95601184, 0.63128161, 0.21370226, 0.93855318, 0.25896422,
       0.78917384, 0.34944675, 0.7950311 , 0.30753868, 0.5858444 ,
       0.97048198, 0.34978868, 0.53048482, 0.30381641, 0.73454347]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])]


In [4]:
@njit()
def p_bias(state,per):
    actions = getValidActions(state)
    actions *= per[0]
    action = np.argmax(actions)
    if getReward(state) == 1:
        per[1] += per[0]
    if getReward(state) == 0:
        per[0] = np.random.rand(getActionSize())
    return action,per

win, bias = numba_main_2(p_bias, 10000,bias1,0)
# Bias tối ưu 
bias_greedy = (bias[1]/np.max(bias[1]))
print(bias_greedy)


[1829.69409367 1922.22825074 1953.99464563 2022.34509348 1789.22846338
 1782.02932609 1770.88381396 1830.42989835 1784.67195658 1767.03420773
 1816.3629286  1838.01724178 1904.32226534 1818.85065626 1784.67235139]


In [24]:
# Check tỷ lệ thắng của bias tối ưu 
@njit()
def p_bias_optim(state,per):
    actions = getValidActions(state)
    actions *= bias_greedy
    action = np.argmax(actions)
    return action,per
win, bias = numba_main_2(p_bias_optim, 1000,bias1,0)
print(win)

5624


#  feature cho các action 0 


In [14]:
#State old là state Hiện tại 
#State new là state Mới 
old = np.load('old0.npy')
new = np.load('new0.npy')
# state mà có cả Value của old and new (1,0)
new = np.load('new0.npy')
o = (new>old)
o= 1*o
i = np.zeros((245142,1))
for u in range(245142) :
    if sum(o[u])>0 :
        # print(o[u])
        i[u] = 1
#  a matrix with random value
l = np.random.uniform(-1,1,(161,1))
#state mà kích hoạt (1,0)
k = 1*((new@l)>=0)
# state mà kích hoạt và có reward (1,0)
y = np.zeros((245142,1))
for p in range(245142):
    if sum(k[p]) >= 1 and sum(i[p])>= 1:
        y[p] = 1
    else:
        y[p] = 0
        
print(sum(y)/sum(k))
# print(old.shape)
print(sum(y))
l = l.reshape(1,161)

[0.9996475]
[201349.]


In [16]:
perx=[0]
@njit
def p0(state,per):
    actions = getValidActions(state)
    matran1 = l@state    
    for i in range(len(matran1)):
        matran1[i] = int(matran1[i] >= 0)
    if matran1 == 1 :
        action = 0 
    else:
        actions = np.where(actions == 1)[0]
        action = actions[np.random.randint(len(actions))]
        while action== 0:
            action = actions[np.random.randint(len(actions))]
    return action,per
win, x = numba_main_2(p0, 1000,perx,0)
print('tỷ lệ thắng',(win/1000)*100)

tỷ lệ thắng 37.4


#  feature cho các action 1


In [17]:
old1 = np.load('old1.npy')
new1 = np.load('new1.npy')
# state mà có cả Value của old and new (1,0)
o1 = (new1>old1)
o1= 1*o1
i1 = np.zeros((245142,1))
for u1 in range(245142) :
    if sum(o1[u1])>0 :
        i1[u1] = 1
#  a matrix with random value
l1 = np.random.uniform(-1,1,(161,1))
#state mà kích hoạt (1,0)
k1 = 1*((new1@l1)>=0)
# state mà kích hoạt và có reward (1,0)
y1 = np.zeros((245142,1))
for p1 in range(245142):
    if sum(k1[p1]) >= 1 and sum(i1[p1])>= 1:
        y1[p1] = 1
    else:
        y1[p1] = 0
print(sum(y1)/sum(k1))
# print(old.shape)
print(sum(y1))
l1 = l1.reshape(1,161)

[0.998856]
[205184.]


In [22]:
@njit
def p_action1(state,per):
    actions = getValidActions(state)
    matran1 = l1@state    
    for i in range(len(matran1)):
        matran1[i] = int(matran1[i] >= 0)
    if matran1 == 1 :
        action = 1
    else:
        actions = np.where(actions == 1)[0]
        action = actions[np.random.randint(len(actions))]
        while action== 1:
            action = actions[np.random.randint(len(actions))]
    return action,per
win, x = numba_main_2(p_action1, 1000,perx,0)
print('tỷ lệ thắng',(win/1000)*100)

tỷ lệ thắng 45.300000000000004


# Ghép 2 action với nhau 

In [28]:
perx1 = [bias_greedy]

0.9504946791405442


In [ ]:
@njit
def p_all_action(state,per):
    actions = getValidActions(state)
    matran1 = l1@state   
    matran2 = l@state    
 
    for i in range(len(matran1)):
        matran1[i] = int(matran1[i] >= 0)
    for i in range(len(matran2)):
        matran2[i] = int(matran2[i] >= 0)

    if matran1 == 1 and matran2 == 0:
        action = 1
    elif matran2 == 1 and matran1 == 0 :
        action = 0
    elif matran2 == matran1 and matran1 == 1:
        if per[0][0]+1 > per[0][1]+1:
            action = 0
        else:
            action = 1
    else:
        actions = np.where(actions == 1)[0]
        action = actions[np.random.randint(len(actions))]
        while action== 1 or action == 0:
            action = actions[np.random.randint(len(actions))]
    return action,per
win, x = numba_main_2(p_all_action, 1000,perx1,0)
print('tỷ lệ thắng',(win/1000)*100)

# Homework 1:

*  **1.For each action , save at least 200k state both old and new state** 
*  **2.Predict what value in state will change from 0 to 1** 
*  **3.Label the state index which change** 



In [7]:
@njit()
def p_bias(state,per):
    actions = getValidActions(state)
    actions *= per[0]
    action = np.argmax(actions)
    if getReward(state) == 1:
        per[1] += per[0]
    if getReward(state) == 0:
        per[0] = np.random.rand(getActionSize())
    return action,per

win, bias = numba_main_2(p_bias, 1,bias1,0)
# Bias tối ưu 
bias_greedy = (bias[1]/np.max(bias[1]))
print(bias_greedy)


[0.92078632 0.96461228 1.         0.99014747 0.90232291 0.90178851
 0.89813277 0.90902076 0.920929   0.88228035 0.90348654 0.889742
 0.87964966 0.89754577 0.90426253]


In [81]:
import numpy as np
from base.Splendor_v3.env import *
@njit()
def p0(state,per):
  # Choose an action
  actions = getValidActions(state)
  actions = np.where(actions == 1)[0]
  action = actions[np.random.randint(len(actions))]
  # Lưu new state
  if  per[3][0][0] == 1: 
    if int(per[1][0][0])<200001:
      j = int(per[1][0][0])
      # print('state sau khi action ',state)
      for i in range(getStateSize()):
        per[2][j][i] += state[i]
      # print('lưu state sau khi action ',per[2][j])
      # trigger action 2
      per[3][0][0] = 0
      print("action sau khi action 2",action)
  #Lưu old state 
  if action == 2 :
    if int(per[1][0][0])<200001:
      # save state
      j = int(per[1][0][0])
      # print('state trước khi action ',state)
      for i in range(getStateSize()):
        per[0][j][i] += state[i]
      # print('lưu state trước khi action ',per[0][j])
      # count number of state
      per[1][0][0] += 1
      per[3][0][0] = 1
      print("action ",action)
    
  return action,per


perx = [np.zeros((200000,getStateSize())),np.zeros((1,1)),np.zeros((200000,getStateSize())),np.zeros((1,1))]
          #state old                         #số trận         #state new                      # trigger
win, x = numba_main_2(p0, 1000000,perx , 0)
np.save('old2.npy',x[0])
np.save('new2.npy',x[2])

action  2
action sau khi action 2 2
action  2
action sau khi action 2 2
action  2
action sau khi action 2 6
action  2
action sau khi action 2 10
action  2
action sau khi action 2 1
action  2
action sau khi action 2 10
action  2
action sau khi action 2 5
action  2
action sau khi action 2 13
action  2
action sau khi action 2 13
action  2
action sau khi action 2 14
action  2
action sau khi action 2 7
action  2
action sau khi action 2 5
action  2
action sau khi action 2 1
action  2
action sau khi action 2 5
action  2
action sau khi action 2 10
action  2
action sau khi action 2 10
action  2
action sau khi action 2 1
action  2
action sau khi action 2 14
action  2
action sau khi action 2 4
action  2
action sau khi action 2 6
action  2
action sau khi action 2 13
action  2
action sau khi action 2 3
action  2
action sau khi action 2 1
action  2
action sau khi action 2 2
action  2
action sau khi action 2 14
action  2
action sau khi action 2 0
action  2
action sau khi action 2 4
action  2
action s

In [83]:
k = np.load('old2.npy')
print(k-x[0])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


# Homework 2:
**Tìm bộ feature cho từng action sao cho**
* **(tỷ lệ state vừa kích hoạt vừa có value /tỷ lệ state vừa kích hoạt) luôn bằng 1**
* **check xem từng action tỷ lệ thắng có trên 30%**

# Homework 3:
**1 agent tổng quán sử dụng bộ feature và bias có tỷ lệ thắng trên 60%**